
Note that the model was fine tuned on Google Colabs, this is the respective file for reference.

In [ ]:
# Install Libraries
!pip install transformers datasets accelerate -q
!pip install pandas scikit-learn -q
!pip install numpy==1.26.4 --q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00


In [ ]:
# Load and Prepare Dataset
import pandas as pd
from datasets import Dataset

df = pd.read_csv("mental_health_dataset.csv")
dataset = Dataset.from_pandas(df)

# Map text labels (e.g., Anxiety) to integers
label2id = {label: idx for idx, label in enumerate(sorted(df["status"].unique()))}
id2label = {v: k for k, v in label2id.items()}

df = df.dropna(subset=["statement", "status"])

In [ ]:
# Add numeric label column
df["label"] = df["status"].map(label2id)

# Prepare Hugging Face dataset
dataset = Dataset.from_pandas(df[["statement", "label"]])


In [ ]:
# Tokenize using Roberta tokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer

model_ckpt = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_ckpt)

def preprocess_function(examples):
    # Ensure all statements are strings and not None
    texts = [str(x) if x is not None else "" for x in examples["statement"]]
    return tokenizer(texts, truncation=True, padding=True)

tokenized_dataset = dataset.train_test_split(test_size=0.2)
tokenized_dataset = tokenized_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/42144 [00:00<?, ? examples/s]

Map:   0%|          | 0/10537 [00:00<?, ? examples/s]

In [ ]:
# Load and configure model
model = RobertaForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# TrainingArguments + Trainer
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average='weighted')
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./roberta-mental-health",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True  # only if using GPU
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-9-3583753791.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lishaangral (lishaangral-netaji-subhas-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.445800,0.454554,0.836576,0.837729
2,0.352100,0.442276,0.847680,0.846358
3,0.253000,0.524967,0.860492,0.860641


TrainOutput(global_step=15804, training_loss=0.39116942034706653, metrics={'train_runtime': 4037.346, 'train_samples_per_second': 31.316, 'train_steps_per_second': 3.914, 'total_flos': 3.32671503458304e+16, 'train_loss': 0.39116942034706653, 'epoch': 3.0})

In [ ]:
# Save the tuned model
tokenizer.save_pretrained("./roberta-mental-health")
model.save_pretrained("./roberta-mental-health")

In [ ]:
!pip install nltk
import nltk
import os

# Loading tokenizer
tokenizer = RobertaTokenizer.from_pretrained("./roberta-mental-health")

# Downloading punkt tokenizer
nltk.download('punkt_tab')

# Reading the book
with open("./Diagnostic and Statistical Manual of Mental Disorders (5th ed.).txt", "r", encoding="utf-8") as f:
    text = f.read()

# Split into sentences (to avoid cutting mid-sentence)
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(text)

# Join short sentences for better context
block_size = 512
inputs = tokenizer(sentences, return_special_tokens_mask=True, truncation=True, padding="max_length", max_length=block_size)

# Create Hugging Face Dataset
dataset2 = Dataset.from_dict(inputs)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM.from_pretrained("./roberta-mental-health")

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at ./roberta-mental-health and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="../models/roberta-mental-health-v2",  # Save new version
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    eval_strategy="no",
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=100
)

In [ ]:
# Data Collator for MLM
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [ ]:
# Train again on Psychology book

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset2,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/tmp/ipython-input-24-3739531732.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,11.076700
200,6.677100
300,5.071400
400,4.412400
500,4.200000
600,3.853600
700,3.601000
800,3.483800
900,3.356700
1000,3.245600


TrainOutput(global_step=7491, training_loss=2.4205393416272702, metrics={'train_runtime': 3062.8121, 'train_samples_per_second': 19.565, 'train_steps_per_second': 2.446, 'total_flos': 1.57761828880128e+16, 'train_loss': 2.4205393416272702, 'epoch': 3.0})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
# Load your fine-tuned model
model = RobertaForSequenceClassification.from_pretrained("../models/roberta-mental-health-v2")
tokenizer = RobertaTokenizer.from_pretrained("../models/roberta-mental-health-v2")

# Define repo name
repo_name = "lishaangral/roberta-mental-health"

# Push model and tokenizer to Hugging Face Hub
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)